<a href="https://colab.research.google.com/github/Daimond92/TelecomX2-data-science/blob/develop/Telecom_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importaciones

In [45]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from os import truncate
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


#📌 Extracción y Preparación de Datos

## Extracción

In [35]:
# Leer el archivo CSV directamente desde la URL
url = "https://raw.githubusercontent.com/Daimond92/TelecomX2-data-science/refs/heads/develop/Data/Telecom_X.csv"
df = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
df.head()

,ID_Cliente,Evasion,Genero,Es_Senior,¿Cliente con pareja?,¿Cliente con dependientes?,Tiempo_con_la_empresa,¿Servicio_Telefono?,¿Varias_Lineas_Telefono?,Servicio_Internet,...,¿Proteccion_Dispositivos?,¿Soporte_Tecnico_Internet?,¿TV_Streaming?,¿Peliculas_Streaming?,Tipo_Contrato,¿Factura_Electronica?,Metodo_Pago,Facturacion_Mensual,Facturacion_Total,Cuentas_Diarias
0,2,0.0,0,0,1,1,9,1,0,0,...,0,1,1,0,1,1,1,65.6,593.30,2.186667
1,3,0.0,1,0,0,0,9,1,1,0,...,0,0,0,1,0,0,1,59.9,542.40,1.996667
2,4,1.0,1,0,0,0,4,1,0,1,...,1,0,0,0,0,1,0,73.9,280.85,2.463333
3,11,1.0,1,1,1,0,13,1,0,1,...,1,0,1,1,0,1,0,98.0,1237.85,3.266667
4,13,1.0,0,1,1,0,3,1,0,1,...,0,1,1,0,0,1,1,83.9,267.40,2.796667


## Preparación de datos

### Eliminar columnas innecesarias

In [36]:
df = df.drop(columns=['ID_Cliente'])


## One-Hot Encoding

In [37]:
# Aplicar One-Hot Encoding a 'Tipo_Contrato'
df_encoded = pd.get_dummies(df, columns=['Tipo_Contrato'], drop_first=True)

# Aplicar One-Hot Encoding a 'Servicio_Internet'
df_encoded = pd.get_dummies(df_encoded, columns=['Servicio_Internet'], drop_first=True)

# Aplicar One-Hot Encoding a 'Metodo_Pago'
df_encoded = pd.get_dummies(df_encoded, columns=['Metodo_Pago'], drop_first=True)

# Ver las primeras filas del dataframe transformado
df_encoded.head()

,Evasion,Genero,Es_Senior,¿Cliente con pareja?,¿Cliente con dependientes?,Tiempo_con_la_empresa,¿Servicio_Telefono?,¿Varias_Lineas_Telefono?,¿Seguridad_Online?,¿Copia_de_Seguridad_Online?,...,Facturacion_Mensual,Facturacion_Total,Cuentas_Diarias,Tipo_Contrato_1,Tipo_Contrato_2,Servicio_Internet_1,Servicio_Internet_2,Metodo_Pago_1,Metodo_Pago_2,Metodo_Pago_3
0,0.0,0,0,1,1,9,1,0,0,1,...,65.6,593.30,2.186667,True,False,False,False,True,False,False
1,0.0,1,0,0,0,9,1,1,0,0,...,59.9,542.40,1.996667,False,False,False,False,True,False,False
2,1.0,1,0,0,0,4,1,0,0,0,...,73.9,280.85,2.463333,False,False,True,False,False,False,False
3,1.0,1,1,1,0,13,1,0,0,1,...,98.0,1237.85,3.266667,False,False,True,False,False,False,False
4,1.0,0,1,1,0,3,1,0,0,0,...,83.9,267.40,2.796667,False,False,True,False,True,False,False


In [34]:
# Convertir todas las columnas booleanas a 0 y 1
df_encoded = df_encoded.astype(int)

# Verificar las primeras filas del dataframe transformado
df_encoded.head()

,ID_Cliente,Evasion,Genero,Es_Senior,¿Cliente con pareja?,¿Cliente con dependientes?,Tiempo_con_la_empresa,¿Servicio_Telefono?,¿Varias_Lineas_Telefono?,¿Seguridad_Online?,...,Facturacion_Mensual,Facturacion_Total,Cuentas_Diarias,Tipo_Contrato_1,Tipo_Contrato_2,Servicio_Internet_1,Servicio_Internet_2,Metodo_Pago_1,Metodo_Pago_2,Metodo_Pago_3
0,2,0,0,0,1,1,9,1,0,0,...,65,593,2,1,0,0,0,1,0,0
1,3,0,1,0,0,0,9,1,1,0,...,59,542,1,0,0,0,0,1,0,0
2,4,1,1,0,0,0,4,1,0,0,...,73,280,2,0,0,1,0,0,0,0
3,11,1,1,1,1,0,13,1,0,0,...,98,1237,3,0,0,1,0,0,0,0
4,13,1,0,1,1,0,3,1,0,0,...,83,267,2,0,0,1,0,1,0,0


## Proporción cancelados (Churn)

In [38]:
# Contar los valores únicos de la columna 'Evasion' (0 y 1)
cancelados = df_encoded['Evasion'].value_counts()

# Calcular la proporción de cancelados vs activos
proporcion_cancelados = cancelados[1] / cancelados.sum()  # Proporción de cancelados
proporcion_activos = cancelados[0] / cancelados.sum()  # Proporción de activos

# Mostrar resultados
print(f"Proporción de clientes que cancelaron: {proporcion_cancelados:.2f}")
print(f"Proporción de clientes activos: {proporcion_activos:.2f}")

# Mostrar el conteo de cada clase
cancelados

Proporción de clientes que cancelaron: 0.27
Proporción de clientes activos: 0.73


,count
Evasion,
0.0,5163
1.0,1869


## Balanceo de datos

In [42]:
# Separar las variables independientes (X) y dependientes (y)
X = df_encoded.drop('Evasion', axis=1)
y = df_encoded['Evasion']

In [43]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Aplicar SMOTE para generar ejemplos sintéticos en el conjunto de entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Ver la nueva distribución de clases en el conjunto de entrenamiento
print(f"Distribución después de SMOTE en entrenamiento: \n{y_train_resampled.value_counts()}")

Distribución después de SMOTE en entrenamiento: 
Evasion
0.0    4130
1.0    4130
Name: count, dtype: int64


## Estandarización y normalización de datos

In [46]:
# Crear el objeto para estandarización
scaler = StandardScaler()

In [47]:
# Aplicar la estandarización en los datos
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

In [48]:
# Crear el objeto para normalización
scaler = MinMaxScaler()

In [49]:
# Aplicar la normalización en los datos
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Crear y entrenar el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_resampled, y_train_resampled)

# Crear y entrenar el modelo XGBoost
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_resampled, y_train_resampled)

# Evaluar los modelos en el conjunto de prueba
rf_pred = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

# Imprimir los resultados de evaluación
print("Random Forest - Clasificación:\n", classification_report(y_test, rf_pred))
print(f"Random Forest - AUC-ROC: {roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]):.2f}")

print("XGBoost - Clasificación:\n", classification_report(y_test, xgb_pred))
print(f"XGBoost - AUC-ROC: {roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:, 1]):.2f}")

In [ ]:
from sklearn.svm import SVC

# Crear y entrenar el modelo SVM
svm_model = SVC(probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train_resampled)

# Evaluar el modelo en el conjunto de prueba
svm_pred = svm_model.predict(X_test_scaled)

# Imprimir los resultados de evaluación
print("SVM - Clasificación:\n", classification_report(y_test, svm_pred))
print(f"SVM - AUC-ROC: {roc_auc_score(y_test, svm_model.predict_proba(X_test_scaled)[:, 1]):.2f}")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Crear y entrenar el modelo KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_scaled, y_train_resampled)

# Evaluar el modelo en el conjunto de prueba
knn_pred = knn_model.predict(X_test_scaled)

# Imprimir los resultados de evaluación
print("KNN - Clasificación:\n", classification_report(y_test, knn_pred))
print(f"KNN - AUC-ROC: {roc_auc_score(y_test, knn_model.predict_proba(X_test_scaled)[:, 1]):.2f}")


In [ ]:
from sklearn.neural_network import MLPClassifier

# Crear y entrenar el modelo de Redes Neuronales
nn_model = MLPClassifier(random_state=42, max_iter=1000)
nn_model.fit(X_train_scaled, y_train_resampled)

# Evaluar el modelo en el conjunto de prueba
nn_pred = nn_model.predict(X_test_scaled)

# Imprimir los resultados de evaluación
print("Redes Neuronales - Clasificación:\n", classification_report(y_test, nn_pred))
print(f"Redes Neuronales - AUC-ROC: {roc_auc_score(y_test, nn_model.predict_proba(X_test_scaled)[:, 1]):.2f}")


#🔧 Transformación de Datos

#📊 Carga y análisis de datos

#📄Informe final Telecom